Write code to classify two famous celebrities:
We need to prepare the data to do so and also prepare the model as well.

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
class MyCLRuleMonitor(tf.keras.callbacks.Callback):
  def __init__(self, CL):
    super(MyCLRuleMonitor).__init__()
    self.CL = CL

  def on_epoch_end(self, epoch, logs=None):
    trainScore = logs["accuracy"]
    testScore = logs["val_accuracy"]

    if testScore > trainScore and testScore >= self.CL:
      self.model.stop_training = True

Steps for Model creation:
0. Data Preprocessing
    i. Image Generators
    ii. Splitting Train and Test data from Generators
1. Model architecting + Prepro
    i. CNN + ANN layers creation
2. Model Compilation
3. Model Training 
4. Model Evaluation
5. Model Testing / Deploy

In [4]:
# ImageGenerators:
training_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)
testing_image_generator= tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1.0/255.0)

In [6]:
training_image_data = training_image_generator.flow_from_directory('celeb/data/training',
                                                                   batch_size = 20,
                                                                   class_mode = 'binary',
                                                                   target_size= (64,64))

testing_image_data = training_image_generator.flow_from_directory('celeb/data/validation',
                                                                   batch_size = 20,
                                                                   class_mode = 'binary',
                                                                   target_size= (64,64))

Found 31 images belonging to 2 classes.
Found 22 images belonging to 2 classes.


In [ ]:
# Model creation:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(32, (3,3), input_shape = training_image_data.image_shape , activation = 'relu', padding = 'same'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

model.add(tf.keras.layers.Conv2D(32, (3,3), input_shape = training_image_data.image_shape, activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

model.add(tf.keras.layers.Flatten())

# FC:
model.add(tf.keras.layers.Dense(units = 128, activation='relu'))
model.add(tf.keras.layers.Dense(units = 128, activation='relu'))
model.add(tf.keras.layers.Dense(units = 128, activation='relu'))
model.add(tf.keras.layers.Dense(units = 128, activation='relu'))

model.add(tf.keras.layers.Dense(units = 1, activation='sigmoid'))

model.summary()

c:\Users\thedo\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
